In [106]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [107]:
#jwddf = pd.read_csv('D:/data/sun/dbscan/jwd_all.csv')
#jwddf = pd.read_csv('D:/data/sun/dbscan/jwd_all.csv')
jwddf = pd.read_csv('D:/data/sun/dbscan/out1.csv')

xqwz = pd.DataFrame({'xq_code':'','east_longitude':'','west_longitude':'','north_latitude':'','south_latitude':''},pd.Index(range(1)))
xq_code_list = []
pre_night_longitude_max_list = []
pre_night_longitude_min_list = []
pre_night_latitude_max_list = []
pre_night_latitude_min_list = []

In [108]:
def showAllPointImage(jwd,xq_code):
    plt.scatter(jwd['pre_night_longitude'], jwd['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def showCalPointImage(test_df,xq_code):
    plt.scatter(test_df['pre_night_longitude'], test_df['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def showClusterImage(core_member,non_core_member,noise_member,n_clusters_):
    plt.plot(core_member['pre_night_longitude'], core_member['pre_night_latitude'], '+', markersize=6)
    plt.plot(non_core_member['pre_night_longitude'], non_core_member['pre_night_latitude'], '^', markersize=6)
    plt.plot(noise_member['pre_night_longitude'], noise_member['pre_night_latitude'], '.', markersize=6)
    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()
    
def showMemberImage(xy,xq_code):
    plt.scatter(xy['pre_night_longitude'], xy['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def calMedian(jwd):
    longitude_zws = jwd['pre_night_longitude'].median()
    latitude_zws = jwd['pre_night_latitude'].median()
    return longitude_zws,latitude_zws
    
def calMean(jwd):
    longitude_pjs = jwd['pre_night_longitude'].mean()
    latitude_pjs = jwd['pre_night_latitude'].mean()
    return longitude_pjs,latitude_pjs

def filterExcepValues(jwd,xq_code):
    longitude_zws,latitude_zws = calMedian(jwd)
    test_df = jwd[(jwd.pre_night_longitude > longitude_zws - 0.025) &
        (jwd.pre_night_longitude < longitude_zws + 0.025) &
        (jwd.pre_night_latitude > latitude_zws - 0.025) &
        (jwd.pre_night_latitude < latitude_zws + 0.025)]
    
    excep_num = jwd.shape[0] - test_df.shape[0]
    print('xq_code：',xq_code ,':全部数据点：',jwd.shape[0],'个：可用数据点：',test_df.shape[0],'个：异常数据点:',excep_num,'个')
    return test_df

def dbscan(test_df,eps1,min_samples1):
    db = DBSCAN(eps=eps1, min_samples=min_samples1).fit(test_df[['pre_night_longitude','pre_night_latitude']])
    labels = db.labels_
    #计算簇个数
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    return db,n_clusters_,labels

def call(df,eps):
    i = 0
    jwddf_groupby = df.groupby(by = 'xq_code')
    xq_codes = list(jwddf_groupby.groups.keys())
    otherdf = pd.DataFrame()
    for xq_code in xq_codes:
        i += 1
        print("当前计算第",i,"个小区")
        jwd = df.loc[df['xq_code']==xq_code]
        index = range(df.shape[0])

        #展示全部用户位置分布
        #showAllPointImage(jwd,xq_code)

        #剔除异常值
        test_df = filterExcepValues(jwd,xq_code)

        #展示参与计算的用户的位置分布
        #showCalPointImage(test_df,xq_code)

        if test_df.shape[0] != 0:
            #db,n_clusters_,labels = dbscan(test_df,eps1,round(test_df.shape[0] * 0.4))
            db,n_clusters_,labels = dbscan(test_df,eps,round(test_df.shape[0] * 0.4))

            if n_clusters_ == 1:
                core_samples_mask = np.zeros_like(labels, dtype=bool)
                #核心点
                core_samples_mask[db.core_sample_indices_] = True
                #成员点
                class_member_mask = (labels == 0)
                #噪声点
                noise_member_mask = (labels == -1)

                core_member = test_df[class_member_mask & core_samples_mask]
                non_core_member = test_df[class_member_mask & ~core_samples_mask]
                class_member = test_df[class_member_mask]
                noise_member = test_df[noise_member_mask]


                #展示簇分布
                #showClusterImage(core_member,non_core_member,noise_member,n_clusters_)
                #展示成员点分布
                #showMemberImage(class_member,xq_code)

                #print('xq_code：',xq_code,',计算点数量：',test_df.shape[0],',成员点数量：',class_member.shape[0],',核心点数量：',core_member.shape[0],',非核心点数量：',non_core_member.shape[0],',噪音点数量：',noise_member.shape[0])
                #print('xq_code：',xq_code,class_member['pre_night_longitude'].max(),class_member['pre_night_longitude'].min(),class_member['pre_night_latitude'].max(),class_member['pre_night_latitude'].min()) 

                east = class_member['pre_night_longitude'].max()
                west = class_member['pre_night_longitude'].min()
                north = class_member['pre_night_latitude'].max()
                south = class_member['pre_night_latitude'].min()

                if (east - west) > 0.0005 and (north - south) > 0.0005:
                    xq_code_list.append(xq_code)
                    pre_night_longitude_max_list.append(east)
                    pre_night_longitude_min_list.append(west)
                    pre_night_latitude_max_list.append(north)
                    pre_night_latitude_min_list.append(south)
                else:
                    #print(df[df.xq_code == xq_code])
                    otherdf = otherdf.append(df[df.xq_code == xq_code],ignore_index=True)
                    #pass
                    print('xq_code：',xq_code ,'小区范围大小，重新计算')

            else:
                otherdf = otherdf.append(df[df.xq_code == xq_code],ignore_index=True)
                #pass
                print('xq_code：',xq_code ,'计算簇个数为',n_clusters_,'个，稍后计算！')

        else:
            pass
            #print('xq_code：',xq_code ,':可用数据点为',test_df.shape[0],'个，稍后计算！')
    
    print('otherdf.shape',otherdf.shape)
    print(otherdf)
    
    if otherdf.shape[0] != 0:
        eps += 0.001
        print('eps=',eps)
        if eps <= 0.01:
            call(otherdf,eps)
        else:
           return
    else:
        return

In [109]:
call(jwddf,0.001)

当前计算第 1 个小区
xq_code： BUSINESS_COMMUNITY-0000000054995ac90154aa61edf80042 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-0000000054995ac90154aa61edf80042 小区范围大小，重新计算
当前计算第 2 个小区
xq_code： BUSINESS_COMMUNITY-141450066916 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450066916 小区范围大小，重新计算
当前计算第 3 个小区
xq_code： BUSINESS_COMMUNITY-141450067347 :全部数据点： 30 个：可用数据点： 24 个：异常数据点: 6 个
xq_code： BUSINESS_COMMUNITY-141450067347 计算簇个数为 0 个，稍后计算！
当前计算第 4 个小区
xq_code： BUSINESS_COMMUNITY-141450067362 :全部数据点： 34 个：可用数据点： 20 个：异常数据点: 14 个
xq_code： BUSINESS_COMMUNITY-141450067362 计算簇个数为 0 个，稍后计算！
当前计算第 5 个小区
xq_code： BUSINESS_COMMUNITY-141450067369 :全部数据点： 26 个：可用数据点： 23 个：异常数据点: 3 个
xq_code： BUSINESS_COMMUNITY-141450067369 小区范围大小，重新计算
当前计算第 6 个小区
xq_code： BUSINESS_COMMUNITY-141450067372 :全部数据点： 18 个：可用数据点： 12 个：异常数据点: 6 个
xq_code： BUSINESS_COMMUNITY-141450067372 计算簇个数为 0 个，稍后计算！
当前计算第 7 个小区
xq_code： BUSINESS_COMMUNITY-141450067384 :全部数据点： 26 个：可用数据点： 20 个：异常数据点: 6 个
xq_code

xq_code： BUSINESS_COMMUNITY-141450071304 计算簇个数为 2 个，稍后计算！
当前计算第 71 个小区
xq_code： BUSINESS_COMMUNITY-141450071352 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450071352 小区范围大小，重新计算
当前计算第 72 个小区
xq_code： BUSINESS_COMMUNITY-141450071360 :全部数据点： 5 个：可用数据点： 0 个：异常数据点: 5 个
当前计算第 73 个小区
xq_code： BUSINESS_COMMUNITY-141450071472 :全部数据点： 6 个：可用数据点： 0 个：异常数据点: 6 个
当前计算第 74 个小区
xq_code： BUSINESS_COMMUNITY-141450071505 :全部数据点： 3 个：可用数据点： 2 个：异常数据点: 1 个
xq_code： BUSINESS_COMMUNITY-141450071505 计算簇个数为 2 个，稍后计算！
当前计算第 75 个小区
xq_code： BUSINESS_COMMUNITY-141450071561 :全部数据点： 7 个：可用数据点： 2 个：异常数据点: 5 个
xq_code： BUSINESS_COMMUNITY-141450071561 计算簇个数为 2 个，稍后计算！
当前计算第 76 个小区
xq_code： BUSINESS_COMMUNITY-141450071595 :全部数据点： 4 个：可用数据点： 0 个：异常数据点: 4 个
当前计算第 77 个小区
xq_code： BUSINESS_COMMUNITY-141450071656 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 78 个小区
xq_code： BUSINESS_COMMUNITY-141450071670 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450071670 小区范围大小，重新计算
当前计算第 79 个小区
xq_

当前计算第 149 个小区
xq_code： BUSINESS_COMMUNITY-141450857130 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 150 个小区
xq_code： BUSINESS_COMMUNITY-141450857152 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450857152 小区范围大小，重新计算
当前计算第 151 个小区
xq_code： BUSINESS_COMMUNITY-141450857160 :全部数据点： 4 个：可用数据点： 0 个：异常数据点: 4 个
当前计算第 152 个小区
xq_code： BUSINESS_COMMUNITY-141450857344 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450857344 小区范围大小，重新计算
当前计算第 153 个小区
xq_code： BUSINESS_COMMUNITY-141450857379 :全部数据点： 5 个：可用数据点： 1 个：异常数据点: 4 个
xq_code： BUSINESS_COMMUNITY-141450857379 小区范围大小，重新计算
当前计算第 154 个小区
xq_code： BUSINESS_COMMUNITY-141450857484 :全部数据点： 5 个：可用数据点： 0 个：异常数据点: 5 个
当前计算第 155 个小区
xq_code： BUSINESS_COMMUNITY-141450857485 :全部数据点： 2 个：可用数据点： 2 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450857485 计算簇个数为 2 个，稍后计算！
当前计算第 156 个小区
xq_code： BUSINESS_COMMUNITY-141450857489 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 157 个小区
xq_code： BUSINESS_COMMUNITY-141450857493 :全部数据点： 1 个：可用数据

xq_code： BUSINESS_COMMUNITY-141452975995 :全部数据点： 5 个：可用数据点： 3 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141452975995 计算簇个数为 3 个，稍后计算！
当前计算第 218 个小区
xq_code： BUSINESS_COMMUNITY-141452976010 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141452976010 小区范围大小，重新计算
当前计算第 219 个小区
xq_code： BUSINESS_COMMUNITY-141452976403 :全部数据点： 15 个：可用数据点： 2 个：异常数据点: 13 个
xq_code： BUSINESS_COMMUNITY-141452976403 计算簇个数为 2 个，稍后计算！
当前计算第 220 个小区
xq_code： BUSINESS_COMMUNITY-141452976473 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141452976473 小区范围大小，重新计算
当前计算第 221 个小区
xq_code： BUSINESS_COMMUNITY-141452976553 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 222 个小区
xq_code： BUSINESS_COMMUNITY-141452976557 :全部数据点： 5 个：可用数据点： 2 个：异常数据点: 3 个
xq_code： BUSINESS_COMMUNITY-141452976557 计算簇个数为 2 个，稍后计算！
当前计算第 223 个小区
xq_code： BUSINESS_COMMUNITY-141452976664 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141452976664 小区范围大小，重新计算
当前计算第 224 个小区
xq_code： BUSINESS_COMMUNITY-141452976689 :

xq_code： BUSINESS_COMMUNITY-141457800803 小区范围大小，重新计算
当前计算第 287 个小区
xq_code： BUSINESS_COMMUNITY-141457800846 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 288 个小区
xq_code： BUSINESS_COMMUNITY-141457800849 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141457800849 小区范围大小，重新计算
当前计算第 289 个小区
xq_code： BUSINESS_COMMUNITY-141457800852 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 290 个小区
xq_code： BUSINESS_COMMUNITY-141457800856 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 291 个小区
xq_code： BUSINESS_COMMUNITY-141457800858 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141457800858 小区范围大小，重新计算
当前计算第 292 个小区
xq_code： BUSINESS_COMMUNITY-141457869544 :全部数据点： 4 个：可用数据点： 0 个：异常数据点: 4 个
当前计算第 293 个小区
xq_code： BUSINESS_COMMUNITY-141457869548 :全部数据点： 7 个：可用数据点： 2 个：异常数据点: 5 个
xq_code： BUSINESS_COMMUNITY-141457869548 计算簇个数为 2 个，稍后计算！
当前计算第 294 个小区
xq_code： BUSINESS_COMMUNITY-141457869564 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141457869564 小区范围大小，重新计算
当前计算第 295 个小区
xq_c

...

xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123154a01029 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123154a01029 小区范围大小，重新计算
当前计算第 542 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123154c41041 :全部数据点： 10 个：可用数据点： 2 个：异常数据点: 8 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123154c41041 计算簇个数为 2 个，稍后计算！
当前计算第 543 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123155041079 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d123155041079 小区范围大小，重新计算
当前计算第 544 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d311773e01439 :全部数据点： 3 个：可用数据点： 2 个：异常数据点: 1 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d311773e01439 小区范围大小，重新计算
当前计算第 545 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d363dd35715c9 :全部数据点： 7 个：可用数据点： 2 个：异常数据点: 5 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105cbfcb8f015d363dd35715c9 计算簇个数为 2 个，稍后计算！
当前计算第 546 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105d50a8dc015d55cf3a710900 :全部数据点： 1 个：可用数据点：

xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ec8591edd4891 小区范围大小，重新计算
当前计算第 607 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ec859220049e1 :全部数据点： 4 个：可用数据点： 1 个：异常数据点: 3 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ec859220049e1 小区范围大小，重新计算
当前计算第 608 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ec85922104a09 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ec85922104a09 小区范围大小，重新计算
当前计算第 609 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7fa9594b09 :全部数据点： 13 个：可用数据点： 1 个：异常数据点: 12 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7fa9594b09 小区范围大小，重新计算
当前计算第 610 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7fae784d52 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7fae784d52 小区范围大小，重新计算
当前计算第 611 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7faee44dc2 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4015ecd7faee44dc2 小区范围大小，重新计算
当前计算第 612 个小区
x

当前计算第 672 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab40160ea3aff112f4a :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab40160ea3aff112f4a 小区范围大小，重新计算
当前计算第 673 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab401610e4778af30aa :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab401610e4778af30aa 小区范围大小，重新计算
当前计算第 674 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4016118942f47313a :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4016118942f47313a 小区范围大小，重新计算
当前计算第 675 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab401611dba8ecd3162 :全部数据点： 2 个：可用数据点： 2 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab401611dba8ecd3162 计算簇个数为 2 个，稍后计算！
当前计算第 676 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4016122e0e4f53273 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4016122e0e4f53273 小区范围大小，重新计算
当前计算第 677 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d105dfa5ab4016146ed6918353b :全部数据点： 1 

xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb016486d149c20912 :全部数据点： 4 个：可用数据点： 1 个：异常数据点: 3 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb016486d149c20912 小区范围大小，重新计算
当前计算第 739 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb016486e91343093a :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb016486e91343093a 小区范围大小，重新计算
当前计算第 740 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebb56700d0a :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebb56700d0a 小区范围大小，重新计算
当前计算第 741 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebc91240d32 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebc91240d32 小区范围大小，重新计算
当前计算第 742 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebd31b40d4a :全部数据点： 2 个：可用数据点： 2 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ebd31b40d4a 计算簇个数为 2 个，稍后计算！
当前计算第 743 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01648ec350ac0d7a :全部数据点： 2 个：可用数据点： 2 个：异

xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166189dcc212b18 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166189dcc212b18 小区范围大小，重新计算
当前计算第 804 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01661dfad3752fa8 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01661dfad3752fa8 小区范围大小，重新计算
当前计算第 805 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016632493b073b78 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016632493b073b78 小区范围大小，重新计算
当前计算第 806 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016675c8f8704c28 :全部数据点： 3 个：可用数据点： 3 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016675c8f8704c28 计算簇个数为 3 个，稍后计算！
当前计算第 807 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01669e95be605b08 :全部数据点： 11 个：可用数据点： 2 个：异常数据点: 9 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01669e95be605b08 计算簇个数为 2 个，稍后计算！
当前计算第 808 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a0143b7f5c60 :全部数据点： 1 个：可用数据点：

In [110]:
xqwz = pd.DataFrame(xq_code_list)
xqwz['east_longitude'] = pd.DataFrame(pre_night_longitude_max_list)
xqwz['west_longitude'] = pd.DataFrame(pre_night_longitude_min_list)
xqwz['north_latitude'] = pd.DataFrame(pre_night_latitude_max_list)
xqwz['south_latitude'] = pd.DataFrame(pre_night_latitude_min_list)

In [111]:
xqwz.to_csv('D:/data/sun/dbscan/xqwz.csv')
print('end')

end
